## Analiza dokładności i czasu uczenia dla Big Data - porównanie metody gradientu prostego, metody gradientu stochastycznego i zmniejszenia zbioru uczącego.

Imię, nazwisko: Krzysztof Jarek.

Kierunek, semestr: IT II, 2.

Data oddania: 24.01.2023.


## Cele:

* implementacja funkcji dokonującej dobrej binaryzacji dla celów znajdowania okręgów,
* implementacja transformaty Hough'a dla celu znajdowania okręgów.

## Przebieg:

Wykorzystane oprogramowanie: Google Colaboratory

Podjęty problemem polega na tym by można było rozpoznawać kształty okręgów na obrazach. W tym celu w ramach projektu uwaga została zwrócona na transformatę Hough'a- czy też pewne rozwiązanie korzystające z niej- dedykowaną dla przypadku rozpoznawania kształtów kolistych.

Zanim jednak przystąpi się do implementacji samej transformaty, najpierw należy zbudować konieczne narzędzie do odpowiedniej obróbki zdjęć tak by mogły się nadawać do dalszej, właściwej analizy.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [2]:
spam_df = pd.read_csv('data/spam.data', header=None, sep=' ')

In [3]:
print( 'Rozmiar zbioru:', spam_df.shape )

Rozmiar zbioru: (4601, 58)


In [4]:
spam_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
5,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.00,0.223,0.0,0.000,0.000,0.000,3.000,15,54,1
6,0.00,0.00,0.00,0.0,1.92,0.00,0.00,0.00,0.00,0.64,...,0.00,0.054,0.0,0.164,0.054,0.000,1.671,4,112,1
7,0.00,0.00,0.00,0.0,1.88,0.00,0.00,1.88,0.00,0.00,...,0.00,0.206,0.0,0.000,0.000,0.000,2.450,11,49,1
8,0.15,0.00,0.46,0.0,0.61,0.00,0.30,0.00,0.92,0.76,...,0.00,0.271,0.0,0.181,0.203,0.022,9.744,445,1257,1
9,0.06,0.12,0.77,0.0,0.19,0.32,0.38,0.00,0.06,0.00,...,0.04,0.030,0.0,0.244,0.081,0.000,1.729,43,749,1


In [5]:
spam_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [6]:
X = spam_df.iloc[:, :-1].values
y = spam_df.iloc[:, -1].values

In [7]:
print( 'Rozmiar zbiorów X i y:', X.shape, 'i', y.shape, '.' )
print( 'Typ danych dla zbiorów X i y:', X.dtype, 'i', y.dtype, '.' )

Rozmiar zbiorów X i y: (4601, 57) i (4601,) .
Typ danych dla zbiorów X i y: float64 i int64 .


In [8]:
X_norm = np.copy(X)
X_mean, X_std = X_norm.mean(), X_norm.std()
X_norm = ( X_norm - X_mean ) / X_std

In [107]:
# def sigmoid(z):
#     return 1.0 / (1 + np.exp(-z))
def sigmoid(x):  
    return np.exp(-np.logaddexp(0.0000000000000001, -x))

In [108]:
def CostFunction(X, y, theta):
    
    m = y.shape[0];
    #h_theta = 1 / (1 + np.exp( -np.dot(theta, X.T) ))
    z = np.dot(theta, X.T)
    h_theta = sigmoid(z)
    
    J  = - ( y.T.dot( np.log(h_theta) ) + (1 - y).T.dot( np.log(1-h_theta) ) ) /m
    dJ = (h_theta - y).T.dot(X) /m
    
    return J, dJ

In [109]:
def GradientDescent( X, y, in_theta, alpha=0.1, num_iter=50 ):
    theta = in_theta.copy()
    
    for _ in range(num_iter):
        J, dJ = CostFunction(X, y, theta)
        theta = theta - alpha * dJ
        
    return theta, J

In [122]:
ALPHA = 0.01
NUM_ITER = 400

In [123]:
m, n  = X.shape
theta = np.random.rand(n)

In [124]:
thetaOpt, JOpt = GradientDescent(X_norm, y, theta, ALPHA, NUM_ITER)

In [125]:
JOpt

0.6540171027728048

In [16]:
# function PlotBoundry( X, Y, Theta, mu, sig)

# figure;
# PlotData( X .* sig + mu, Y );

# T = unique( X(2,:) * sig(2) + mu(2));
# t = unique( X(3,:) * sig(3) + mu(3));
# y = Theta' * X;
# y = reshape(y, length(t), length(T));

# contour( T, t, y, [0, 0], 'k');

# legend( 'y = 1', 'y = 0', 'line', 'Location', 'northoutside' );
# xlabel('T, C');
# ylabel('t, s');

# end